In [ ]:
#Training the Teacher Model (LSTM) on full dataset.

In [ ]:
#Imports

In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Data

In [ ]:
forecast_df = pd.read_csv("forecasted.csv")
obs_df = pd.read_csv("observeddata.csv")
forecast_df['Forecast_Date'] = pd.to_datetime(forecast_df['Forecast_Date'])
obs_df['Date'] = pd.to_datetime(obs_df['Date'])
grid_points = forecast_df[['Lat', 'Lon']].drop_duplicates().values.tolist()
first_issue_date = pd.to_datetime('2003-06-07')

In [ ]:
def get_lead_time(forecast_date):
    delta_days = (forecast_date - first_issue_date).days
    cycles = delta_days // 7
    issue_date = first_issue_date + pd.Timedelta(days=7*cycles)
    return (forecast_date - issue_date).days + 1

output_leads = {lt: [] for lt in range(1, 8)}

In [ ]:
#Modelling

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
all_data = []

for lat, lon in grid_points:
    fcst_grid = forecast_df[(forecast_df['Lat'] == lat) & (forecast_df['Lon'] == lon)].copy()
    obs_grid = obs_df[(obs_df['Lat'] == lat) & (obs_df['Lon'] == lon)].copy()

    pivot = fcst_grid.pivot_table(index='Forecast_Date', columns='Model', values='Precipitation').reset_index()
    pivot.rename(columns={'Forecast_Date': 'Date'}, inplace=True)

    merged = pd.merge(pivot, obs_grid, on='Date', how='inner')
    merged.sort_values("Date", inplace=True)

    model_cols = [col for col in merged.columns if col.startswith(("CFST", "GFST"))]
    for col in model_cols:
        merged[f"{col}_residual"] = merged[col] - merged["Observed_Precip"]

    merged["Ensemble_Mean"] = merged[model_cols].mean(axis=1)
    merged["Ensemble_Spread"] = merged[model_cols].std(axis=1)
    merged["Ensemble_Residual"] = merged["Ensemble_Mean"] - merged["Observed_Precip"]

    merged["Obs_RollMean_3"] = merged["Observed_Precip"].rolling(window=3).mean()
    merged["Obs_RollStd_3"] = merged["Observed_Precip"].rolling(window=3).std()
    merged["Obs_RollMean_7"] = merged["Observed_Precip"].rolling(window=7).mean()
    merged["Obs_RollStd_7"] = merged["Observed_Precip"].rolling(window=7).std()

    merged["DOY"] = merged["Date"].dt.dayofyear
    climatology = merged.groupby("DOY")["Observed_Precip"].mean()
    merged["Climatology"] = merged["DOY"].map(climatology)
    merged["Anomaly"] = merged["Observed_Precip"] - merged["Climatology"]

    merged = merged.iloc[7:].reset_index(drop=True)
    merged["Lead_Time"] = merged["Date"].apply(get_lead_time)
    merged["Lat"] = lat
    merged["Lon"] = lon

    all_data.append(merged)

df_all = pd.concat(all_data).reset_index(drop=True)

for lead in range(1, 8):
    df_lead = df_all[df_all["Lead_Time"] == lead].copy()
    if df_lead.empty:
        continue

    model_cols = [col for col in df_lead.columns if col.startswith(("CFST", "GFST"))]
    model_resid_cols = [f"{col}_residual" for col in model_cols]

    feature_cols = model_cols + model_resid_cols + [
        "Ensemble_Mean", "Ensemble_Spread", "Climatology", "Anomaly",
        "Obs_RollMean_3", "Obs_RollStd_3", "Obs_RollMean_7", "Obs_RollStd_7", "DOY"
    ]
    
    X = df_lead[feature_cols].values
    y = df_lead["Ensemble_Residual"].values.reshape(-1, 1)

    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # (samples, time_steps, features)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)

    input_layer = Input(shape=(1, X_train.shape[2]))
    x = Conv1D(filters=32, kernel_size=1, activation='relu')(input_layer)
    x = LSTM(64, activation='relu', return_sequences=True)(x)
    x = Dropout(0.2)(x)
    x = LSTM(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    output = Dense(1)(x)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=64, verbose=1)

    y_pred = model.predict(X_test).flatten()
    y_pred_inv = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()

    df_out = df_lead.iloc[-len(y_test_inv):].copy()
    df_out["New_Residual"] = y_pred_inv
    df_out["New_Prediction"] = df_out["Ensemble_Mean"] - df_out["New_Residual"]

    final_cols = ['Date', 'Observed_Precip', 'Ensemble_Mean', 'Ensemble_Residual', 'New_Residual', 'New_Prediction', 'Lat', 'Lon']
    df_out[final_cols].to_csv(f"lead{lead}.csv", index=False)

In [ ]:
#End